In [179]:
import sys
import csv
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
import time
import pandas as pd

In [193]:
# Initializing global parameters
# I initialize and close the driver separately for each airline due to changes in the DOM

num_pages = 100
airlines_list = ["air-france","klm","ryanair","hop","emirates","lufthansa","swiss-intl-air","austrian-airlines","british-airways","virgin-atlantic-airways","aeroflot","iberia","turkish-airlines"]


In [194]:
urls = {
        "air-france":"https://www.tripadvisor.com/Airline_Review-d8729003-Reviews-Air-France",
        "klm":"https://www.tripadvisor.com/Airline_Review-d8729104-Reviews-KLM-Royal-Dutch-Airlines",
        "ryanair":"https://www.tripadvisor.com/Airline_Review-d8729141-Reviews-Ryanair",
        "hop":"https://www.tripadvisor.com/Airline_Review-d8728906-Reviews-HOP",
        "emirates":"https://www.tripadvisor.com/Airline_Review-d8729069-Reviews-Emirates",
        "lufthansa":"https://www.tripadvisor.com/Airline_Review-d8729113-Reviews-Lufthansa",
        "swiss-intl-air":"https://www.tripadvisor.com/Airline_Review-d8729160-Reviews-Swiss-International-Air-Lines-SWISS",
        "austrian-airlines":"https://www.tripadvisor.com/Airline_Review-d8729027-Reviews-Austrian-Airlines",
        "british-airways":"https://www.tripadvisor.com/Airline_Review-d8729039-Reviews-British-Airways",
        "virgin-atlantic-airways":"https://www.tripadvisor.com/Airline_Review-d8729182-Reviews-Virgin-Atlantic-Airways",
        "aeroflot":"https://www.tripadvisor.com/Airline_Review-d8728987-Reviews-Aeroflot",
        "iberia":"https://www.tripadvisor.com/Airline_Review-d8729089-Reviews-Iberia",
        "turkish-airlines":"https://www.tripadvisor.com/Airline_Review-d8729174-Reviews-Turkish-Airlines"
       }
        

In [195]:
def get_reviews(airline, num_pages): 
    driver = webdriver.Chrome(executable_path='C:/chromedriver.exe')
    driver.get(urls[airline])
    reviews=[]
    for i in range(num_pages):
        
        #To allow the page to load before the code runs
        time.sleep(2)
        
        #To accept cookies
        try:
            driver.find_element_by_xpath('.//button[@class="evidon-banner-acceptbutton"]').click()
        except:
            pass
        
        #To expand "Read More"
        try:
            driver.find_element_by_xpath("//span[@class='_3maEfNCR']").click()
        except:
            pass
        
        reviews_scraped = driver.find_elements_by_class_name("cPQsENeY")
        
        #To collect all reviews in a page in a list
        for review in reviews_scraped[1:]:
            reviews.append(review.text.replace("\n", "  "))
            
        #To navigate to the next page        
        try:
            driver.find_element_by_xpath('.//a[@class="ui_button nav next primary "]').click()
        except:
            break
        
    driver.close()
    review_df = pd.DataFrame(reviews)
    return review_df

In [196]:
for airline in airlines_list:
    df = get_reviews(airline, num_pages)
    df['airline'] = airline
    df.columns = ['review', 'airline']
    #print(df.head())
    df.to_csv('TripAdvisor_' + airline + '.csv', sep='|', index = False)

In [198]:
import glob
import os
path = r"."                     # use your path
all_files = glob.glob(os.path.join(path, "*.csv"))     # advisable to use os.path.join as this makes concatenation OS independent
print(all_files)
li = []
for filename in all_files:
    print(filename)
    df = pd.read_csv(filename, sep="|")
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

['.\\TripAdvisor_aeroflot.csv', '.\\TripAdvisor_air-france.csv', '.\\TripAdvisor_austrian-airlines.csv', '.\\TripAdvisor_british-airways.csv', '.\\TripAdvisor_emirates.csv', '.\\TripAdvisor_hop.csv', '.\\TripAdvisor_iberia.csv', '.\\TripAdvisor_klm.csv', '.\\TripAdvisor_lufthansa.csv', '.\\TripAdvisor_ryanair.csv', '.\\TripAdvisor_swiss-intl-air.csv', '.\\TripAdvisor_turkish-airlines.csv', '.\\TripAdvisor_virgin-atlantic-airways.csv']
.\TripAdvisor_aeroflot.csv
.\TripAdvisor_air-france.csv
.\TripAdvisor_austrian-airlines.csv
.\TripAdvisor_british-airways.csv
.\TripAdvisor_emirates.csv
.\TripAdvisor_hop.csv
.\TripAdvisor_iberia.csv
.\TripAdvisor_klm.csv
.\TripAdvisor_lufthansa.csv
.\TripAdvisor_ryanair.csv
.\TripAdvisor_swiss-intl-air.csv
.\TripAdvisor_turkish-airlines.csv
.\TripAdvisor_virgin-atlantic-airways.csv


In [200]:
frame.columns = [ 'review', 'airline']
frame.to_csv("TripAdvisor_merged.csv", sep='|')